In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas

1. 멜론차트 시대별 페이지를 연다
2. 곡정보 칸의 아이콘을 클릭하여 곡정보 페이지로 이동한다.
3. 제목과 가수, 가사를 크롤링하여 리스트로 저장한다.
3-1. 펼치기 버튼을 눌러 가사 전체가 보이게 한 후
3-2. 가사를 크롤링한다.
4. 시대별 페이지에서 51-100위 버튼을 클릭하여 이 과정을 반복한다.
5. 이 과정을 2010년부터 2021년까지 반복한다.
5-1. 연도 글자 옆의 꺽쇠 >를 눌러 다음 페이지로 넘어간다.
6. 크롤링한 결과를 pandas로 보인다.
7. 크롤링한 결과를 csv 파일로 저장한다.

In [2]:
#제목, 가수, 가사 크롤링하는 lyricsCrawling 함수 정의

def lyricsCrawling1():
    global melon_lyrics
    for i in range(50):
        btn_list = driver.find_elements_by_xpath('//*[@id="lst50"]/td[4]/div/a/span')
    
        # actionchains를 불러온 후 곡정보 버튼을 눌러 곡정보 페이지로 이동한다.
        actions = webdriver.ActionChains(driver)
        actions.move_to_element(btn_list[i]).pause(1).click().perform()
    
        try:
            # 노래 제목과 가수 텍스트를 가져온다.
            song_title = driver.find_element_by_class_name("song_name").text
            song_singer = driver.find_element_by_class_name("artist").text
    
            # 펼치기 버튼을 눌러서 가사가 전부 보이도록 한다.
            lyrics_btn = driver.find_element_by_xpath('//*[@id="lyricArea"]/button')
            actions = webdriver.ActionChains(driver)
            actions.move_to_element(lyrics_btn).pause(1).click().perform()
    
            # 가사 텍스트를 가져온다.
            song_lyrics = driver.find_element_by_xpath('//*[@id="d_video_summary"]').text
    
            # 연도, 제목, 가수, 가사를 리스트에 저장한다.
            melon_lyrics.append([year, song_title, song_singer, song_lyrics])
        
            # 시대별 50위 페이지로 돌아간다.
            driver.back()
        
        except: # 성인인증이 필요할 경우 시대별 50위 페이지로 돌아간 후 다음 곡부터 반복한다.
            driver.back()
            continue

In [3]:
def lyricsCrawling2():
    global melon_lyrics
    for i in range(50):
        btn_list = driver.find_elements_by_xpath('//*[@id="lst100"]/td[4]/div/a/span')
        #xpath id가 상이하여 서로 다른 함수로 구현
    
        actions = webdriver.ActionChains(driver)
        actions.move_to_element(btn_list[i]).pause(1).click().perform()
    
        try:
            song_title = driver.find_element_by_class_name("song_name").text
            song_singer = driver.find_element_by_class_name("artist").text
    
            lyrics_btn = driver.find_element_by_xpath('//*[@id="lyricArea"]/button')
            actions = webdriver.ActionChains(driver)
            actions.move_to_element(lyrics_btn).pause(1).click().perform()
    
            song_lyrics = driver.find_element_by_xpath('//*[@id="d_video_summary"]').text
    
            melon_lyrics.append([year, song_title, song_singer, song_lyrics])
        
            driver.back()
        
        except:
            driver.back()
            continue

In [4]:
# 51-100위 버튼 클릭하는 함수 pagemove 정의
def pagemove():
    next_btn = driver.find_element_by_xpath('//*[@id="tb_list"]/div[2]/span/a')
    actions = webdriver.ActionChains(driver)
    actions.move_to_element(next_btn).pause(1).click().perform()

In [5]:
# 2010년부터 2021년까지 반복하는 함수 yearchange 정의
def yearchange():
    global year
    while year <= 2021:
        lyricsCrawling1()
        pagemove()
        lyricsCrawling2()
        try:
            year_btn = driver.find_element_by_xpath('//*[@id="conts"]/div[3]/div[3]/button[2]')
            actions = webdriver.ActionChains(driver)
            actions.move_to_element(year_btn).pause(1).click().perform()
        except:
            return
        year += 1

In [6]:
driver = webdriver.Chrome()

melon = 'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate=2010'

driver.get(melon)

In [7]:
#크롤링 실행
melon_lyrics = []
year = 2010
yearchange()

In [10]:
# pandas 데이터프레임으로 보이기
df = pandas.DataFrame(melon_lyrics)
df.head()

,0,1,2,3
0,2010,Bad Girl Good Girl,미쓰에이,You don't know me\n\nYou don't know me\n\nYou ...
1,2010,잔소리 (With 2AM 슬옹),아이유,늦게 다니지좀 마\n\n술은 멀리좀 해봐\n\n열살짜리 애처럼 말을 안듣니\n\n정...
2,2010,죽을 만큼 아파서 (Feat. 멜로우),MC몽,I found the way to\nlet you leave\nI never rea...
3,2010,못해 (Feat. 美),포맨 (4MEN),이제는 밥을 먹어도\n눈물없이는 삼키지 못해\n억지로 먹고 먹어도 속이 늘 허전해\...
4,2010,죽어도 못 보내,2am,어려도 아픈건 똑같아\n세상을 잘 모른다고\n아픈걸 모르진 않아\n괜찮아 질거라고 ...


In [15]:
# csv 파일로 저장하기
df.to_csv("meloncrawling.csv", encoding='utf-8-sig', index=False)